In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv', index_col=0)
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv', index_col=0)
submission_df = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv', index_col=0)
train_df

In [ ]:
train_df.describe().T

In [ ]:
train_df.target.value_counts()

In [ ]:
y = train_df.pop('target')
X = train_df

In [ ]:
from sklearn.preprocessing import MinMaxScaler
my_scaler = MinMaxScaler()
X_scaled = pd.DataFrame(my_scaler.fit_transform(X))
test_scaled = pd.DataFrame(my_scaler.transform(test_df))

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

xgb_clf = XGBClassifier(num_class=4, objective='multi:softprob', n_jobs=-1)

params = {
    'model__n_estimators': [20, 50, 100, 200],
    'model__max_depth': [3, 5, 9],
    'model__learning_rate': [0.001, 0.01, 0.1]
}

rs_cv = RandomizedSearchCV(estimator=xgb_clf, cv=3, n_jobs=-1, n_iter=3, scoring='accuracy', verbose=3, param_distributions=params)

In [ ]:
# rs_cv.fit(X_scaled, y)

In [ ]:
# rs_cv.best_params_

In [ ]:
xgb_modified_clf = XGBClassifier(num_class=4, xgb_clfbjective='multi:softprob', n_jobs=-1, n_estimators=100, max_depth=5, learning_rate=0.01)
xgb_modified_clf.fit(X_scaled, y)
y_pred = xgb_modified_clf.predict_proba(test_scaled)

In [ ]:
submission_df['Class_1'] = y_pred[:, 0]
submission_df['Class_2'] = y_pred[:, 1]
submission_df['Class_3'] = y_pred[:, 2]
submission_df['Class_4'] = y_pred[:, 3]

submission_df.to_csv('submission.csv')